# data preproccessing


## import modules and configure notebook

In [1]:
import pandas as pd
import numpy as np
import swifter
import seaborn as sns
import matplotlib.pyplot

pd.set_option('max.rows', None)
pd.set_option('max.columns', None)

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import log_loss, accuracy_score, f1_score
from sklearn.decomposition import PCA 
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler


%matplotlib inline

/home/robert/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### Configurations

In [5]:
data_input_path = '../../data/AllData_2_All_Details_3_OutliersRem_1_SupDep_Regions.csv'

### Import data

In [6]:
my_data = pd.read_csv(data_input_path)

In [12]:
group_superficial = False

In [7]:
my_data['Site'].unique()

array(['FH', 'ER', 'WW', 'TC', 'CS', 'BC', 'KQ', 'AR', 'SL', 'FG', 'WB',
       'BX', 'PF', 'BM', 'WH', 'SQ', 'BP', 'WN', 'BH', 'PH', 'LB', 'AB',
       'LV', 'BR', 'KY', 'BF', 'ST', 'SH', 'CF', 'BG', 'AC', 'CR', 'GH',
       'PX', 'WF', 'DH', 'NMAG_Gold', 'NMW_Gold', 'NMWGwern', 'UBSS',
       'Cefn', 'Stockley', 'Pucha', 'Woodbury', 'Pimple', 'Wellington',
       'Lyonshall', 'SymondsYatE', 'Madawg'], dtype=object)

### make labels for classification

In [8]:

def make_classes_grouped(row):
    if row['Geology'] == 'Bedrock':
        if row['Site'] == 'WB' or row['Site'] == 'BX':
            return('WB_BX')
        elif row['Site'] == 'BC' or row['Site'] == 'CS':
            return('BC_CS')
        elif row['Site'] == 'SQ' or row['Site'] == 'BP':
            return('SQ_BP')
        else:
            return(row['Site'])
    elif row['Geology'] == 'Superficial':
        if row['Region'] == 'SV' or row['Region'] == 'SE':
            return('SV_SE')
        else:
            return(row['Region'])

def make_classes_raw(row):
    if row['Geology'] == 'Bedrock':
        return(row['Site'])
    elif row['Geology'] == 'Superficial':
        return(row['Region'])



In [14]:
my_data.head()

,Analysis,Geology,Province,Region,Site,SubSite,Formation,Band,Nodule,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238,class
0,10_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,15.63,0.12,48.36,154.63,943.71,464944.18,50.28,538.57,455.94,712.39,0.42,15.58,0.27,3.3,0.69,8.46,0.05,0.8,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.51,0.09,0.05,0.02,0,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0,0.04,0.01,0.24,0.07,0.05,FH
1,11_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,11.50,0.09,44.77,NaN,1077.11,465010.94,70.91,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,<0.10,0.45,13.22,0.95,1.74,0.07,<0.01,<0.02,<0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0,0.07,0.08,0.04,FH
2,12_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,20.05,0.06,44.88,42.70,620.21,465295.41,104.47,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.93,0.1,0.02,<0.02,<0.00,0.05,0.01,3.13,0.9,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0,0.02,0.01,0.46,0.05,0.05,FH
3,13_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,11.16,<0.73,47.06,162.42,1143.19,NaN,56367.93,1075.89,547.55,2174.3,<0.43,42.3,0.67,152.42,4.84,145.34,0.3,2.45,5.02,17.15,0.35,2.13,<0.84,0.76,13.16,0.97,2.00,0.1,0.29,<0.18,<0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0,0.03,<0.00,0.08,<0.00,0.64,0.05,0.03,FH
4,14_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,17.71,0.32,48.26,33.52,547.22,465027.11,44.44,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.8,0.55,9.8,0.41,1.41,<0.12,0.28,9.90,0.9,0.90,0.08,0.04,0.1,0,0.09,0.01,2.74,0.97,1.09,0.27,1,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09,FH


In [13]:
my_data['class'] = 'init'   

if group_superficial:
    my_data['class'] = my_data.apply(make_classes_grouped, axis = 1)
else:
    my_data['class'] = my_data.apply(make_classes_raw, axis = 1)

### Impute values that contaion '<' sign with 0 because the sign represents a measurement below the detection liits of the instrument

In [18]:
for column_name in my_data.columns.values[9:-1]:
    def fill_less_than(row):
        if '<' in str(row[column_name]):
            return(float(row[column_name].replace('<', '').replace(',','')))
        else:
            return(float(row[column_name]))
    my_data[column_name] = my_data.swifter.apply(fill_less_than, axis = 1)

Pandas Apply: 100%|██████████| 1606/1606 [00:00<00:00, 19153.38it/s]


### Impute na values with variable median, this is more resistant to the effect of outliers

In [19]:
for column_name in my_data.columns.values[9:-1]:
    my_data[column_name] = my_data[column_name].fillna(my_data[column_name].median()) 

In [20]:
my_data.head()

,Analysis,Geology,Province,Region,Site,SubSite,Formation,Band,Nodule,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238,class
0,10_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,15.63,0.12,48.36,154.63,943.71,464944.18,50.28,538.57,455.94,712.39,0.42,15.58,0.27,3.30,0.69,8.46,0.05,0.80,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.51,0.09,0.05,0.02,0.00,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0.00,0.04,0.01,0.24,0.07,0.05,FH
1,11_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,11.50,0.09,44.77,22.42,1077.11,465010.94,70.91,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,0.10,0.45,13.22,0.95,1.74,0.07,0.01,0.02,0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0.00,0.07,0.08,0.04,FH
2,12_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,20.05,0.06,44.88,42.70,620.21,465295.41,104.47,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.93,0.10,0.02,0.02,0.00,0.05,0.01,3.13,0.90,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0.00,0.02,0.01,0.46,0.05,0.05,FH
3,13_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,11.16,0.73,47.06,162.42,1143.19,465099.89,56367.93,1075.89,547.55,2174.30,0.43,42.30,0.67,152.42,4.84,145.34,0.30,2.45,5.02,17.15,0.35,2.13,0.84,0.76,13.16,0.97,2.00,0.10,0.29,0.18,0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0.00,0.03,0.00,0.08,0.00,0.64,0.05,0.03,FH
4,14_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,17.71,0.32,48.26,33.52,547.22,465027.11,44.44,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.80,0.55,9.80,0.41,1.41,0.12,0.28,9.90,0.90,0.90,0.08,0.04,0.10,0.00,0.09,0.01,2.74,0.97,1.09,0.27,1.00,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09,FH


In [24]:
my_data.isna().any()

Analysis     False
Geology      False
Province      True
Region        True
Site         False
SubSite       True
Formation     True
Band          True
Nodule       False
Li7          False
Be9          False
B11          False
Mg24         False
Al27         False
Si28         False
P31          False
S33          False
K39          False
Ca42         False
Sc45         False
Ti47         False
V51          False
Cr52         False
Mn55         False
Fe56         False
Co59         False
Ni60         False
Cu63         False
Zn68         False
Ga69         False
Ge72         False
As75         False
Rb85         False
Sr88         False
Y89          False
Zr90         False
Nb93         False
Mo95         False
Cd111        False
In115        False
Sn118        False
Cs133        False
Ba137        False
La139        False
Ce140        False
Pr141        False
Nd146        False
Sm147        False
Eu153        False
Gd157        False
Tb159        False
Dy163        False
Ho165       

### Filter for known data bedrock data for training the model

In [29]:
train_data = my_data[(my_data['Geology']== 'Bedrock') | (my_data['Geology'] == 'Superficial')]
test_data = my_data[my_data['Geology']=='Artefacts']


In [30]:
my_data.shape

(1606, 63)

In [31]:
train_data.shape

(1243, 63)

In [32]:
test_data.shape

(363, 63)

### label encode the class to be predicted

In [33]:
train_data_formodel = train_data.copy(deep = True)
train_data_formodel['class'], uniques = pd.factorize(train_data_formodel['class'])


In [45]:
train_data.head(100)

,Analysis,Geology,Province,Region,Site,SubSite,Formation,Band,Nodule,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238,class
0,10_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,15.63,0.12,48.36,154.63,943.71,464944.18,50.280,538.57,455.94,712.39,0.42,15.58,0.27,3.30,0.69,8.46,0.05,0.80,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.510,0.09,0.05,0.02,0.00,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0.00,0.04,0.01,0.24,0.07,0.05,FH
1,11_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,11.50,0.09,44.77,22.42,1077.11,465010.94,70.910,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,0.10,0.45,13.22,0.95,1.740,0.07,0.01,0.02,0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0.00,0.07,0.08,0.04,FH
2,12_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,20.05,0.06,44.88,42.70,620.21,465295.41,104.470,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.930,0.10,0.02,0.02,0.00,0.05,0.01,3.13,0.90,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0.00,0.02,0.01,0.46,0.05,0.05,FH
3,13_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,11.16,0.73,47.06,162.42,1143.19,465099.89,56367.930,1075.89,547.55,2174.30,0.43,42.30,0.67,152.42,4.84,145.34,0.30,2.45,5.02,17.15,0.35,2.13,0.84,0.76,13.16,0.97,2.000,0.10,0.29,0.18,0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0.00,0.03,0.00,0.08,0.00,0.64,0.05,0.03,FH
4,14_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,17.71,0.32,48.26,33.52,547.22,465027.11,44.440,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.80,0.55,9.80,0.41,1.41,0.12,0.28,9.90,0.90,0.900,0.08,0.04,0.10,0.00,0.09,0.01,2.74,0.97,1.09,0.27,1.00,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09,FH
5,15_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,17.27,0.06,47.89,112.27,925.56,465048.10,59.390,496.86,346.89,763.47,0.55,14.75,0.31,3.20,0.86,30.50,0.06,1.04,0.53,9.56,0.23,1.27,0.11,0.42,11.17,0.91,1.680,0.08,0.02,0.03,0.00,0.04,0.01,7.40,0.84,0.92,0.24,0.89,0.19,0.04,0.17,0.02,0.11,0.02,0.06,0.01,0.05,0.00,0.04,0.01,0.11,0.09,0.04,FH
6,16_FH1_1_3,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_3,14.32,0.16,41.79,194.11,1168.05,462663.07,2038.080,458.82,400.01,765.59,0.43,28.88,0.57,8.81,0.81,21.87,0.12,0.47,0.68,10.55,0.44,0.70,0.13,0.53,14.46,1.17,2.090,0.15,0.03,0.03,0.00,0.09,0.02,8.03,1.12,1.20,0.31,1.16,0.22,0.05,0.23,0.03,0.17,0.03,0.08,0.01,0.04,0.01,0.05,0.01,0.19,0.11,0.03,FH
7,17_FH1_1_3,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_3,16.02,0.03,48.06,61.27,785.43,465237.69,43.210,549.56,308.25,678.31,0.57,36.46,0.21,3.62,0.60,17.41,0.04,0.55,0.63,10.72,0.29,1.52,0.12,0.35,9.93,0.81,1.850,0.15,0.02,0.03,0.00,0.07,0.01,5.57,0.76,0.85,0.19,0.76,0.16,0.03,0.12,0.02,0.12,0.02,0.06,0.01,0.04,0.01,0.04,0.01,0.18,0.06,0.07,FH
8,18_FH1_1_3,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_3,19.56,0.15,42.81,87.03,1141.48,464689.32,268.440,437.20,506.68,671.49,0.54,15.23,0.31,2.76,0.84,38.21,0.09,1.37,1.33,12.81,0.28,0.80,0.11,0.63,13.48,0.83,2.070,0.07,0.04,0.05,0.01,0.05,0.01,7.70,0.82,0.96,0.23,0.84,0.15,0.03,0.19,0.02,0.12,0.02,0.05,0.01,0.03,0.00,0.04,0.00,0.31,0.07,0.03,FH
9,23_FH2_1_1,Bedrock,Northern,NaN,FH,FH2,Burnham,FH2_1,FH2_1_1,24.65,0.15,47.34,109.06,708.96,464743.88,98.180,494.03,315.04,1469.53,0.69,21.73,0.50,3.72,1.69,33.57,0.06,0.96,1.02,10.99,0.38,1.83,0.13,0.42,11.89,0.77,2.180,0.12,0.03,0.11,0.01,0.07,0.02,4.65,0.73,0.90,0.20,0.71,0.10,0.04,0.17,0.02,0.09,0.02,0.06,0.01,0.03,0.01,0.05,0.01,0.21,0.09,0.11,FH


In [44]:
train_data_formodel.head(150)

,Analysis,Geology,Province,Region,Site,SubSite,Formation,Band,Nodule,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238,class
0,10_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,15.63,0.12,48.36,154.63,943.71,464944.18,50.280,538.57,455.94,712.39,0.42,15.58,0.27,3.30,0.69,8.46,0.05,0.80,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.510,0.09,0.05,0.02,0.00,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0.00,0.04,0.01,0.24,0.07,0.05,0
1,11_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,11.50,0.09,44.77,22.42,1077.11,465010.94,70.910,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,0.10,0.45,13.22,0.95,1.740,0.07,0.01,0.02,0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0.00,0.07,0.08,0.04,0
2,12_FH1_1_1,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_1,20.05,0.06,44.88,42.70,620.21,465295.41,104.470,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.930,0.10,0.02,0.02,0.00,0.05,0.01,3.13,0.90,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0.00,0.02,0.01,0.46,0.05,0.05,0
3,13_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,11.16,0.73,47.06,162.42,1143.19,465099.89,56367.930,1075.89,547.55,2174.30,0.43,42.30,0.67,152.42,4.84,145.34,0.30,2.45,5.02,17.15,0.35,2.13,0.84,0.76,13.16,0.97,2.000,0.10,0.29,0.18,0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0.00,0.03,0.00,0.08,0.00,0.64,0.05,0.03,0
4,14_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,17.71,0.32,48.26,33.52,547.22,465027.11,44.440,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.80,0.55,9.80,0.41,1.41,0.12,0.28,9.90,0.90,0.900,0.08,0.04,0.10,0.00,0.09,0.01,2.74,0.97,1.09,0.27,1.00,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09,0
5,15_FH1_1_2,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_2,17.27,0.06,47.89,112.27,925.56,465048.10,59.390,496.86,346.89,763.47,0.55,14.75,0.31,3.20,0.86,30.50,0.06,1.04,0.53,9.56,0.23,1.27,0.11,0.42,11.17,0.91,1.680,0.08,0.02,0.03,0.00,0.04,0.01,7.40,0.84,0.92,0.24,0.89,0.19,0.04,0.17,0.02,0.11,0.02,0.06,0.01,0.05,0.00,0.04,0.01,0.11,0.09,0.04,0
6,16_FH1_1_3,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_3,14.32,0.16,41.79,194.11,1168.05,462663.07,2038.080,458.82,400.01,765.59,0.43,28.88,0.57,8.81,0.81,21.87,0.12,0.47,0.68,10.55,0.44,0.70,0.13,0.53,14.46,1.17,2.090,0.15,0.03,0.03,0.00,0.09,0.02,8.03,1.12,1.20,0.31,1.16,0.22,0.05,0.23,0.03,0.17,0.03,0.08,0.01,0.04,0.01,0.05,0.01,0.19,0.11,0.03,0
7,17_FH1_1_3,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_3,16.02,0.03,48.06,61.27,785.43,465237.69,43.210,549.56,308.25,678.31,0.57,36.46,0.21,3.62,0.60,17.41,0.04,0.55,0.63,10.72,0.29,1.52,0.12,0.35,9.93,0.81,1.850,0.15,0.02,0.03,0.00,0.07,0.01,5.57,0.76,0.85,0.19,0.76,0.16,0.03,0.12,0.02,0.12,0.02,0.06,0.01,0.04,0.01,0.04,0.01,0.18,0.06,0.07,0
8,18_FH1_1_3,Bedrock,Northern,NaN,FH,FH1,Burnham,FH1,FH1_1_3,19.56,0.15,42.81,87.03,1141.48,464689.32,268.440,437.20,506.68,671.49,0.54,15.23,0.31,2.76,0.84,38.21,0.09,1.37,1.33,12.81,0.28,0.80,0.11,0.63,13.48,0.83,2.070,0.07,0.04,0.05,0.01,0.05,0.01,7.70,0.82,0.96,0.23,0.84,0.15,0.03,0.19,0.02,0.12,0.02,0.05,0.01,0.03,0.00,0.04,0.00,0.31,0.07,0.03,0
9,23_FH2_1_1,Bedrock,Northern,NaN,FH,FH2,Burnham,FH2_1,FH2_1_1,24.65,0.15,47.34,109.06,708.96,464743.88,98.180,494.03,315.04,1469.53,0.69,21.73,0.50,3.72,1.69,33.57,0.06,0.96,1.02,10.99,0.38,1.83,0.13,0.42,11.89,0.77,2.180,0.12,0.03,0.11,0.01,0.07,0.02,4.65,0.73,0.90,0.20,0.71,0.10,0.04,0.17,0.02,0.09,0.02,0.06,0.01,0.03,0.01,0.05,0.01,0.21,0.09,0.11,0


In [38]:
uniques

Index(['FH', 'ER', 'WW', 'TC', 'CS', 'BC', 'KQ', 'AR', 'SL', 'FG', 'WB', 'BX',
       'PF', 'BM', 'WH', 'SQ', 'BP', 'WN', 'BH', 'PH', 'LB', 'AB', 'LV', 'SE',
       'BA', 'SV', 'WA', 'MM'],
      dtype='object')

### order of class labels as numbers 0 through 20

In [39]:
print(uniques)

Index(['FH', 'ER', 'WW', 'TC', 'CS', 'BC', 'KQ', 'AR', 'SL', 'FG', 'WB', 'BX',
       'PF', 'BM', 'WH', 'SQ', 'BP', 'WN', 'BH', 'PH', 'LB', 'AB', 'LV', 'SE',
       'BA', 'SV', 'WA', 'MM'],
      dtype='object')


In [46]:
%store train_data_formodel
%store train_data
%store test_data
%store my_data
%store uniques

Stored 'train_data_formodel' (DataFrame)
Stored 'train_data' (DataFrame)
Stored 'test_data' (DataFrame)
Stored 'my_data' (DataFrame)
Stored 'uniques' (Index)


In [47]:
train_data_bedrock = train_data[train_data['Geology'] == 'Bedrock']
train_data_superficial = train_data[train_data['Geology'] == 'Superficial']

In [49]:
train_data_bedrock.shape

(808, 63)

In [50]:
train_data_superficial.shape

(435, 63)

In [52]:
train_data.shape

(1243, 63)

In [42]:
train_data_bedrock.columns.values[9:-1]

array(['Li7', 'Be9', 'B11', 'Mg24', 'Al27', 'Si28', 'P31', 'S33', 'K39',
       'Ca42', 'Sc45', 'Ti47', 'V51', 'Cr52', 'Mn55', 'Fe56', 'Co59',
       'Ni60', 'Cu63', 'Zn68', 'Ga69', 'Ge72', 'As75', 'Rb85', 'Sr88',
       'Y89', 'Zr90', 'Nb93', 'Mo95', 'Cd111', 'In115', 'Sn118', 'Cs133',
       'Ba137', 'La139', 'Ce140', 'Pr141', 'Nd146', 'Sm147', 'Eu153',
       'Gd157', 'Tb159', 'Dy163', 'Ho165', 'Er166', 'Tm169', 'Yb172',
       'Lu175', 'Hf178', 'Ta181', 'Pb208', 'Th232', 'U238'], dtype=object)

In [53]:
element_data_train = train_data[train_data.columns.values[9:-1]]
element_data_train_bedrock = train_data_bedrock[train_data.columns.values[9:-1]]
element_data_train_superficial = train_data_superficial[train_data.columns.values[9:-1]]
element_data_test = test_data[test_data.columns.values[9:-1]]

In [54]:
element_data_train.head()

,Li7,Be9,B11,Mg24,Al27,Si28,P31,S33,K39,Ca42,Sc45,Ti47,V51,Cr52,Mn55,Fe56,Co59,Ni60,Cu63,Zn68,Ga69,Ge72,As75,Rb85,Sr88,Y89,Zr90,Nb93,Mo95,Cd111,In115,Sn118,Cs133,Ba137,La139,Ce140,Pr141,Nd146,Sm147,Eu153,Gd157,Tb159,Dy163,Ho165,Er166,Tm169,Yb172,Lu175,Hf178,Ta181,Pb208,Th232,U238
0,15.63,0.12,48.36,154.63,943.71,464944.18,50.28,538.57,455.94,712.39,0.42,15.58,0.27,3.30,0.69,8.46,0.05,0.80,1.62,10.82,0.25,1.22,0.16,0.43,12.94,0.88,1.51,0.09,0.05,0.02,0.00,0.05,0.01,6.54,0.84,0.95,0.23,0.87,0.16,0.04,0.16,0.02,0.11,0.03,0.06,0.01,0.02,0.00,0.04,0.01,0.24,0.07,0.05
1,11.50,0.09,44.77,22.42,1077.11,465010.94,70.91,438.20,387.82,515.24,0.44,18.47,0.29,3.45,1.01,11.59,0.11,0.36,0.53,8.93,0.34,0.85,0.10,0.45,13.22,0.95,1.74,0.07,0.01,0.02,0.00,0.04,0.02,8.04,0.92,1.01,0.23,0.98,0.18,0.04,0.18,0.02,0.13,0.03,0.06,0.01,0.04,0.01,0.05,0.00,0.07,0.08,0.04
2,20.05,0.06,44.88,42.70,620.21,465295.41,104.47,372.66,363.71,957.89,0.76,19.89,0.55,3.25,1.21,87.99,0.21,1.68,1.53,11.98,0.25,1.71,0.13,0.43,8.52,0.87,0.93,0.10,0.02,0.02,0.00,0.05,0.01,3.13,0.90,1.08,0.26,0.84,0.15,0.04,0.19,0.02,0.14,0.02,0.07,0.01,0.06,0.00,0.02,0.01,0.46,0.05,0.05
3,11.16,0.73,47.06,162.42,1143.19,465099.89,56367.93,1075.89,547.55,2174.30,0.43,42.30,0.67,152.42,4.84,145.34,0.30,2.45,5.02,17.15,0.35,2.13,0.84,0.76,13.16,0.97,2.00,0.10,0.29,0.18,0.01,0.78,0.04,8.74,0.93,0.95,0.21,0.75,0.13,0.04,0.25,0.02,0.09,0.03,0.05,0.00,0.03,0.00,0.08,0.00,0.64,0.05,0.03
4,17.71,0.32,48.26,33.52,547.22,465027.11,44.44,464.78,278.25,1551.63,0.71,11.18,0.27,2.56,1.73,25.38,0.05,0.80,0.55,9.80,0.41,1.41,0.12,0.28,9.90,0.90,0.90,0.08,0.04,0.10,0.00,0.09,0.01,2.74,0.97,1.09,0.27,1.00,0.17,0.04,0.19,0.02,0.15,0.03,0.05,0.01,0.05,0.01,0.02,0.01,0.59,0.06,0.09


In [56]:
my_scaler_train = StandardScaler()
my_scaler_train_bedrock = StandardScaler()
my_scaler_train_superficial = StandardScaler()
my_scaler_test = StandardScaler()

In [57]:
element_data_train_scaled = my_scaler_train.fit_transform(element_data_train)
element_data_train_bedrock_scaled = my_scaler_train_bedrock.fit_transform(element_data_train_bedrock)
element_data_train_superficial_scaled = my_scaler_train_superficial.fit_transform(element_data_train_superficial)
element_data_test_scaled = my_scaler_test.fit_transform(element_data_test)

In [62]:
element_data_train_scaled.shape[1]

53

In [63]:
my_pca_train = PCA(n_components=element_data_train_scaled.shape[1])

my_pca_train_bedrock = PCA(n_components=element_data_train_bedrock_scaled.shape[1])
my_pca_train_superficial = PCA(n_components=element_data_train_superficial_scaled.shape[1])

my_pca_test = PCA(n_components=element_data_test_scaled.shape[1])

element_data_train_pca = my_pca_train.fit_transform(element_data_train_scaled)

element_data_train_bedrock_pca = my_pca_train_bedrock.fit_transform(element_data_train_bedrock_scaled)
element_data_train_superficial_pca = my_pca_train_superficial.fit_transform(element_data_train_superficial_scaled)

element_data_test_pca = my_pca_test.fit_transform(element_data_test_scaled)

In [68]:
no_PCs = element_data_train_scaled.shape[1]
PC_names = []
for i in range(0, no_PCs):
    number = i + 1
    column_name = 'PC' + str(number)
    PC_names.append(column_name)
    

In [48]:
PC_df_train = pd.DataFrame(data = element_data_train_pca, columns = PC_names)

PC_df_bedrock_train = pd.DataFrame(data = element_data_train_bedrock_pca, columns = PC_names)
PC_df_superficial_train = pd.DataFrame(data = element_data_train_superficial_pca, columns = PC_names)

PC_df_test = pd.DataFrame(data = element_data_test_pca, columns = PC_names)

In [49]:
my_tsne_train = TSNE(n_components=3, n_iter=750, verbose=3).fit_transform(element_data_train_scaled)

my_tsne_bedrock_train = TSNE(n_components=3, n_iter=750, verbose=3).fit_transform(element_data_train_bedrock_scaled)
my_tsne_superficial_train = TSNE(n_components=3, n_iter=750, verbose=3).fit_transform(element_data_train_superficial_scaled)

my_tsne_test = TSNE(n_components=3, n_iter=750, verbose=3).fit_transform(element_data_test_scaled)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 1243 samples in 0.003s...
[t-SNE] Computed neighbors for 1243 samples in 0.296s...
[t-SNE] Computed conditional probabilities for sample 1000 / 1243
[t-SNE] Computed conditional probabilities for sample 1243 / 1243
[t-SNE] Mean sigma: 1.771276
[t-SNE] Computed conditional probabilities in 0.105s
[t-SNE] Iteration 50: error = 76.8285370, gradient norm = 0.0798455 (50 iterations in 8.038s)
[t-SNE] Iteration 100: error = 78.4137955, gradient norm = 0.0700876 (50 iterations in 6.405s)
[t-SNE] Iteration 150: error = 79.4278259, gradient norm = 0.0689119 (50 iterations in 5.193s)
[t-SNE] Iteration 200: error = 80.1398849, gradient norm = 0.0455261 (50 iterations in 4.922s)
[t-SNE] Iteration 250: error = 80.2874832, gradient norm = 0.0498382 (50 iterations in 4.796s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 80.287483
[t-SNE] Iteration 300: error = 1.7874633, gradient norm = 0.0007194 (50 iterations in 5.267s)

In [50]:
tsne_df_train = pd.DataFrame(data = my_tsne_train, columns = ['tsne1', 'tsne2', 'tsne3'])

tsne_df_bedrock_train = pd.DataFrame(data = my_tsne_bedrock_train, columns = ['tsne1', 'tsne2', 'tsne3'])
tsne_df_superficial_train = pd.DataFrame(data = my_tsne_superficial_train, columns = ['tsne1', 'tsne2', 'tsne3'])

tsne_df_test = pd.DataFrame(data = my_tsne_test, columns = ['tsne1', 'tsne2', 'tsne3'])

In [51]:
%store PC_df_train
%store my_pca_train

%store PC_df_bedrock_train
%store my_pca_train_bedrock

%store PC_df_superficial_train
%store my_pca_train_superficial

%store PC_df_test
%store my_pca_test

%store tsne_df_train

%store tsne_df_bedrock_train
%store tsne_df_superficial_train


%store tsne_df_test


Stored 'PC_df_train' (DataFrame)
Stored 'my_pca_train' (PCA)
Stored 'PC_df_bedrock_train' (DataFrame)
Stored 'my_pca_train_bedrock' (PCA)
Stored 'PC_df_superficial_train' (DataFrame)
Stored 'my_pca_train_superficial' (PCA)
Stored 'PC_df_test' (DataFrame)
Stored 'my_pca_test' (PCA)
Stored 'tsne_df_train' (DataFrame)
Stored 'tsne_df_bedrock_train' (DataFrame)
Stored 'tsne_df_superficial_train' (DataFrame)
Stored 'tsne_df_test' (DataFrame)


In [52]:
import pandas as pd
%store -r tsne_df_train
tsne_df_train.to_csv('tsne_df_train.csv', index = False)
%store -r tsne_df_test
tsne_df_test.to_csv('tsne_df_test.csv', index=False)

%store -r PC_df_train
PC_df_train.to_csv(' PC_df_train.csv', index=False)
%store -r PC_df_test
PC_df_test.to_csv('PC_df_test.csv', index=False)